In [13]:
import mlflow
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline


In [2]:

# Set experiment ui
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

2022/06/20 22:18:33 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [3]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [6]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [11]:
# Custom function not available for use in cross val score
def my_custom_loss_func(y_true, y_pred):
    error = mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False)
    return error

In [21]:
estimators = [100,200,300]
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'n_estimators': hp.choice('n_estimators', [100,200,300]),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf',5, 40, 5)),
    'random_state': 42
}

In [25]:
def objective(params):
    with mlflow.start_run():
        mlflow.log_params(params)

        pipeline = make_pipeline(
            DictVectorizer(),
            RandomForestRegressor(**params, n_jobs=-1)
        )

        cv_scores = cross_val_score(estimator=pipeline, 
                                    X=dict_train, 
                                    y=y_train, 
                                    scoring=make_scorer(my_custom_loss_func, greater_is_better=False), 
                                    cv=3)
                                    
        mean_rmse = np.mean(cv_scores)
        std_rmse = np.std(cv_scores)
        
        rmse = mean_rmse + std_rmse
        mlflow.log_metric('rmse', rmse)
    
    return {'loss': rmse, 'status': STATUS_OK}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)


100%|██████████| 10/10 [05:40<00:00, 34.06s/trial, best loss: -5.542383129660325]


In [26]:
#Extracting best params
best_result['max_depth'] = int(best_result['max_depth'])
best_result['min_samples_leaf'] = int(best_result['min_samples_leaf'])
best_result['n_estimators'] = estimators[best_result['n_estimators']]
best_result['random_state'] = 42

In [27]:
# Lets train the best model
with mlflow.start_run():
    mlflow.set_tag("model", "random-forest") 

    params = best_result
    mlflow.log_params(params)

    pipeline = make_pipeline(
            DictVectorizer(),
            RandomForestRegressor(**params, n_jobs=-1)
        )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    mlflow.log_metric('val_rmse', rmse)
    print(params, rmse)
    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 92, 'min_samples_leaf': 40, 'n_estimators': 200, 'random_state': 42} 6.786923073411247


In [24]:
# Loading the model
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '23f8f015ffc848ddb56fdb8da8e6efc3'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)